## EARLIER EXPERIMENTS

In [1]:
import pandas as pd
import numpy as np
#import networkx as nx
import sys

# for the bag of word features 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# stable numerical implementation of sigmoid
import scipy
from scipy.special import expit

In [2]:
import cvxopt
from cvxopt import matrix, spmatrix, solvers

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
#cosine_similarity(X, Y)

In [4]:
import time

In [5]:
import Levenshtein

In [6]:
def get_exp_mismatch_matrix(words, _lambda):
    N = len(words)

    exp_mismatch_matrix = np.zeros((N, N))
    for i in range(N):
        exp_mismatch_matrix[i,i] = 1
        for j in range(i+1, N):
            exp_mismatch_matrix[i,j] = _lambda**Levenshtein.hamming(words[i], words[j])
            exp_mismatch_matrix[j,i] = exp_mismatch_matrix[i,j]

    return exp_mismatch_matrix

In [7]:
def load_sequence(kind='X', root='tr', number=3):
    seqs =  [pd.read_csv('./data/%s%s%d.csv'%(kind, root, d)) for d in range(number)]
    
    if kind == 'X':
            df = pd.DataFrame(columns=['Id','seq'])
    else:
            df= pd.DataFrame(columns=['Id','Bound'])
    
    for seq in seqs:
        
        df = df.append(seq, ignore_index=True)
        
    return df

def load_features(root='tr', number=3):
    kind = 'X'
    
    feats =  [np.loadtxt('./data/%s%s%d_mat100.csv'%(kind, root, d)) for d in range(number)]
        
    return  np.vstack((feat for feat in feats))


def getKmers(sequence, size=5):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]


def get_features(dF, size=5, normed=False, rang=(4,4)):
    df = dF.copy()
    
    df['words'] = df.apply(lambda x: getKmers(x['seq'], size=size), axis=1)
    df = df.drop('seq', axis=1)

    texts = list(df['words'])
    for item in range(len(texts)):
        texts[item] = ' '.join(df.iloc[item,1])
    
    if normed:
        cv = TfidfVectorizer(ngram_range=rang)
    else:
        cv = CountVectorizer(ngram_range= rang)
    X = cv.fit_transform(texts)
    return X

def get_mismatch(dF, size=5, thres=2000, normed=False, gamma=0.4):
    df = dF.copy()
    
    df['words'] = df.apply(lambda x: getKmers(x['seq'], size=size), axis=1)
    df = df.drop('seq', axis=1)

    texts = list(df['words'])
    for item in range(len(texts)):
        texts[item] = ' '.join(df.iloc[item,1])
    
    if normed:
        cv = TfidfVectorizer()
    else:
        cv = CountVectorizer()
    X = cv.fit_transform(texts)
    
    words = list(cv.get_feature_names())
    S = get_exp_mismatch_matrix(words, 0.3)
    
    k_xx = X[:thres] @ S @ X[:thres].T
    k_yx = X[thres:] @ S @ X[:thres].T
    
    return k_xx, k_yx

In [8]:
def save_my_kernels(df, dg,  k=0, sizes=[3,4,5,6,7]):
    tt = time.time()
    
    for size in sizes:
        print('Doing size: ', size)
        dF = df.iloc[2000*k:2000*(k+1)].append(dg.iloc[1000*k:1000*(k+1)])
        k_xx, k_yx = get_mismatch(dF, size=size, thres=2000, normed=False)
        np.savetxt('./mykernels/k_xx_%d_%d.txt'%(k, size), k_xx)
        np.savetxt('./mykernels/k_yx_%d_%d.txt'%(k, size), k_yx)


    tt = time.time() - tt
    print('done is %.3f seconds'%(tt/60))

In [9]:
sequences_train = load_sequence(number=3)
sequences_test = load_sequence(number=3, root='te')
labels_train = load_sequence(kind='Y' ,root='tr', number=3)

In [10]:
sequences_test.shape, labels_train.shape, sequences_train.shape

((3000, 2), (6000, 2), (6000, 2))

In [11]:
all_labels = labels_train.Bound.values.astype(int)

In [12]:
sizes =  [3, 4, 5, 6, 7]

In [13]:
YET_TO_SAVE = False

if YET_TO_SAVE:
    save_my_kernels(sequences_train, sequences_test, k=0, sizes=sizes)
    save_my_kernels(sequences_train, sequences_test, k=1, sizes=sizes)
    save_my_kernels(sequences_train, sequences_test, k=2, sizes=sizes)

In [15]:
def build_training(k, sizes=[4,5], normed=False, rang=(4,4), gamma=0.4):
    dataset = pd.DataFrame()
    dataset = dataset.append(sequences_train.iloc[2000*k:2000*(k+1)])
    dataset =  dataset.append(sequences_test.iloc[1000*k:1000*(k+1)])

    K_xx, K_yx = [], []
    
    for size in sizes:
        print('counting for size %d '%size)
        counts = get_features(dataset, size=size, normed=normed, rang=rang)

        counts_train = counts[:2000]
        counts_test = counts[2000:]
        print('computing classical training kernel for size %d '%size)
        k_xx = np.dot(counts_train, counts_train.T).toarray()
        print('computing classical testing kernel for size %d '%size)
        k_yx = np.dot(counts_test, counts_train.T).toarray()
        K_xx.append(k_xx)
        K_yx.append(k_yx)
        
        #k_xx, k_yx = get_mismatch(dataset, size=size, thres=2000, normed=normed, gamma=gamma)
        print('loading special training kernel for size %d '%size)
        k_xx = np.loadtxt('./mykernels/k_xx_%d_%d.txt'%(k, size))
        print('loading special test kernel for size %d '%size)
        k_yx = np.loadtxt('./mykernels/k_yx_%d_%d.txt'%(k, size))
        K_xx.append(k_xx)
        K_yx.append(k_yx)
    
    y_train = all_labels[2000*k:2000*(k+1)]
    
    return np.array(K_xx), np.array(K_yx), y_train

## THE PYTHON CLASS FOR KERNEL LOGISTIC REGRESSION

In [16]:
class MultiKerOpt():
    
    def __init__(self, alpha=0.01, tol=1e-07, degree=2, method='klr', hide=False):
        self.alpha = alpha
        self.tol = tol
        self.degree = degree
        self.method = method
        self.hide  = hide
        
    def scale(self, u, norm):
        if norm=='l1':
            return u/np.sum(u)
        elif norm=='l2':
            return u / np.sqrt(np.sum(u**2))
        else:
            raise Exception('l1 and l2 are the only available norms')
            
    def bound(self, u, u_0, gamma, norm):
        u__ = u - u_0
        u__ = np.abs(self.scale(u__, norm) * gamma)
        return u__ + u_0
    
    def KrrIterate(self, Kernels, y, coef, weights = None):
        K_w = np.sum((Kernels * coef[:, None, None]), axis=0) ** self.degree
        N, D = K_w.shape
        if weights is None:
            c = np.linalg.solve(np.linalg.inv(K_w + self.alpha * np.eye(N, D)), y[:, np.newaxis])
        else:
            W_r = np.diag(np.sqrt(weights))
            A = W_r.dot(K_w).dot(W_r) + self.alpha * np.eye(N,D)
            Y = np.dot(W_r, y[:, np.newaxis])
            x_sol = np.linalg.solve(A, Y)
            c = np.dot(W_r, x_sol)
        return c
    
    def KlrIterate(self, Kernels, y, coef, tol=1e-07, max_iters=5):
        c_old = self.KrrIterate(Kernels, y, coef)
        K_w = np.sum((Kernels * coef[:, None, None]), axis=0) ** self.degree
        y_enc = 2*y-1
        
        for i in range(max_iters):
            m_t = np.dot(K_w, c_old)
            p_t = -expit(-y_enc[:, np.newaxis]*m_t)
            w_t = expit(m_t)*expit(-m_t)
            z_t = m_t - (p_t * y_enc[:, np.newaxis]) /(w_t+ 1e-05)
            c_new = self.KrrIterate(Kernels, z_t.flatten(), coef, weights=w_t.flatten())
            if np.linalg.norm(c_new - c_old)<tol:
                break
            else:
                c_old = c_new
        return c_old

    def SvmIterate(self, Kernels, y, coef):
        nb_samples = y.shape[0]
        C = 1 / ( 2 * self.alpha * nb_samples)
        
        r = np.arange(nb_samples)
        o = np.ones(nb_samples)
        z = np.zeros(nb_samples)
            
        K_w  = np.sum(Kernels * coef[:, None, None], axis=0) ** (self.degree)
        
        y_enc = 2*y-1
        
        P = matrix(K_w.astype(float), tc='d')
        q = matrix(-y_enc, tc='d')
        G = spmatrix(np.r_[y_enc, -y_enc], np.r_[r, r + nb_samples], np.r_[r, r], tc='d')
        h = matrix(np.r_[o * C, z], tc='d')
        
        if self.hide:
            solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h)
        c = np.ravel(sol['x'])[:,np.newaxis]
        
        return c
    
    def gradUpdate(self, Kernels, coef, delta):
        K_t = np.sum(Kernels * coef[:, None, None], axis=0) ** (self.degree-1)
        grad = np.zeros(len(Kernels))
        for m in range(len(Kernels)):
            grad[m] = delta.T.dot((K_t * Kernels[m])).dot(delta)
            
        return - self.degree * grad
    
    def fit(self, Kernels, y, u_0=0, gamma=1, norm='l2', n_iter=5, step=1, weights=None):
        coef = np.random.normal(0, 1, len(Kernels)) / len(Kernels)
        coef = self.bound(coef, u_0, gamma, norm)
        new_coef = 0
        
        score_prev = np.inf
        
        for i in range(n_iter):
            #print(i+1)
            if self.method=='klr':
                delta = self.KlrIterate(Kernels, y, coef, tol=1e-07, max_iters=5)
            elif self.method=='svm':
                delta = self.SvmIterate(Kernels, y, coef)
            else:
                delta = self.KrrIterate(Kernels, y, coef, weights = weights)
                
            grad = self.gradUpdate(Kernels, coef, delta)
            
            new_coef = coef - step * grad
            new_coef = self.bound(new_coef, u_0, gamma, norm)
            
            score = np.linalg.norm(new_coef - coef, np.inf)
            
            if score>score_prev:
                step *= 0.9
                
            if score<self.tol:
                self.coef = coef
                self.delta = delta
            
            coef = new_coef
            score_prev = score.copy()
            
        self.coef, self.delta = coef, delta
        #return new_coef
    def predict(self, Kernels):
        K_w = np.sum(Kernels * self.coef[:, None, None], axis=0) ** (self.degree)
        y__ = np.sign(K_w.dot(self.delta)).flatten()
        if self.method != 'krr':
            y__ = 0.5 * (y__ + 1)
        return y__
    
    def score(self, Kernels, y):
        y__ = self.predict(Kernels)
        if self.method!='krr':
            score = 100*(y__==y).mean()
        else:
            score = np.mean((y__- y)**2)
        return score
                

In [17]:
def CvSearch(K_xx, K_yx, y, method='svm', degrees=[4], alphas=[0.01], cv=5):
    tt = time.time()
    
    n_iters = cv * len(degrees) * len(alphas)
    
    n_samples = y.shape[0]
    
    DEG, ALPH, TRAIN, VAL = [], [], [], []
    
    i=0
    
    for degree in degrees:
        for alpha in alphas:
            DEG.append(degree)
            ALPH.append(alpha)
            
            #SPLITTING
            INDS = np.array(range(n_samples))
            idx = np.random.permutation(n_samples)
            INDS = INDS[idx]
            
            vals = np.array_split(INDS, cv)
            
            perfs_train = []
            perfs_val = []
            
            for val in vals:
                i += 1 
                sys.stderr.write('\rIteration %d/%d -- degree %d --alpha %.3f' %(i, n_iters, degree, alpha))
                sys.stderr.flush()
                
                train = np.setdiff1d(range(n_samples),val)
                
                clf = MultiKerOpt(alpha=alpha, tol=1e-07, degree=degree, method=method, hide=True)
                
                clf.fit(K_xx[:,train.reshape(-1,1), train], y[train])
                
                score_train = clf.score(K_xx[:,train.reshape(-1,1), train], y[train])
                
                score_val =  clf.score(K_xx[:,val.reshape(-1,1), train], y[val])
                
                perfs_train.append(score_train)
                perfs_val.append(score_val)
                
            TRAIN.append(np.mean(np.array(perfs_train)))
            VAL.append(np.mean(np.array(perfs_val)))
            
    df = pd.DataFrame({'degree':DEG, 'alpha':ALPH, 'train':TRAIN, 'val':VAL})
    
    tt = time.time() - tt
    print('Done in %.3f'%(tt/60))
    
    return df
#

In [18]:
def get_best(df):
    idx = np.argmax(df.val.values)
    best = np.max(df.val.values)

    best_degree = df.degree[idx]
    best_alpha = df.alpha[idx]
    return best_degree, best_alpha, best

In [19]:
REG_PARAMS_SPAN = [10**i for i in range(-3, 2)] #+ [10**i/2 for i in range(-10, 10)]
print(REG_PARAMS_SPAN)

[0.001, 0.01, 0.1, 1, 10]


## DATASET 0

In [20]:
K_xx_0, X_yx_0, y_train_0 = build_training(0, sizes=[4,5,6,7], normed=False, rang=(4,4))

counting for size 4 
computing classical training kernel for size 4 
computing classical testing kernel for size 4 
loading special training kernel for size 4 
loading special test kernel for size 4 
counting for size 5 
computing classical training kernel for size 5 
computing classical testing kernel for size 5 
loading special training kernel for size 5 
loading special test kernel for size 5 
counting for size 6 
computing classical training kernel for size 6 
computing classical testing kernel for size 6 
loading special training kernel for size 6 
loading special test kernel for size 6 
counting for size 7 
computing classical training kernel for size 7 
computing classical testing kernel for size 7 
loading special training kernel for size 7 
loading special test kernel for size 7 


In [21]:
print(K_xx_0.shape, X_yx_0.shape, y_train_0.shape)

(8, 2000, 2000) (8, 1000, 2000) (2000,)


In [22]:
RUN = False

if RUN:
    df  = CvSearch(K_xx_0, X_yx_0, y_train_0, method='svm', degrees=[1, 2, 3], 
               alphas=REG_PARAMS_SPAN, cv=5)
    df.to_csv('./CV/X0_cv5_linear_kmers_mis.csv', index=False)
else:
    df = pd.read_csv('./CV/X0_cv5_linear_kmers_mis.csv')

Iteration 75/75 

Done in 10.482


In [23]:
df

,degree,alpha,train,val
0,1,0.001,100.0000,62.40
1,1,0.010,100.0000,62.50
2,1,0.100,94.1875,62.60
3,1,1.000,56.5750,51.85
4,1,10.000,51.1625,51.15
5,2,0.001,100.0000,63.25
6,2,0.010,100.0000,63.35
7,2,0.100,100.0000,62.15
8,2,1.000,100.0000,62.25
9,2,10.000,100.0000,61.50


In [24]:
best_degree_0, best_alpha_0, best_0 = get_best(df)
print(best_degree_0, best_alpha_0, best_0)

3 0.01 63.7


In [34]:
clf0 = MultiKerOpt(alpha=best_alpha_0, tol=1e-07, degree=best_degree_0, method='svm', hide=False)
clf0.fit(K_xx_0, y_train_0, n_iter=5)
y_pred_0 = clf0.predict(X_yx_0)

In [35]:
print(y_train_0.mean(), y_pred_0.mean())

0.4885 0.488


## DATASET 1

In [25]:
K_xx_1, X_yx_1, y_train_1 = build_training(1, sizes=[4,5,6], normed=False, rang=(4,4))

counting for size 4 
computing classical training kernel for size 4 
computing classical testing kernel for size 4 
loading special training kernel for size 4 
loading special test kernel for size 4 
counting for size 5 
computing classical training kernel for size 5 
computing classical testing kernel for size 5 
loading special training kernel for size 5 
loading special test kernel for size 5 
counting for size 6 
computing classical training kernel for size 6 
computing classical testing kernel for size 6 
loading special training kernel for size 6 
loading special test kernel for size 6 


In [26]:
RUN = False

if RUN:
    df1  = CvSearch(K_xx_1, X_yx_1, y_train_1, method='svm', degrees=[1, 2, 3], 
               alphas=REG_PARAMS_SPAN, cv=5)
    df1.to_csv('./CV/X1_cv5_linear_kmers_mis.csv', index=False)
else:
    df1 = pd.read_csv('./CV/X1_cv5_linear_kmers_mis.csv')

Iteration 75/75 

Done in 8.930


In [27]:
df1

,degree,alpha,train,val
0,1,0.001,100.0000,74.45
1,1,0.010,100.0000,75.25
2,1,0.100,94.3875,74.20
3,1,1.000,68.5500,63.40
4,1,10.000,57.8625,52.50
5,2,0.001,100.0000,75.65
6,2,0.010,100.0000,74.75
7,2,0.100,100.0000,74.50
8,2,1.000,100.0000,75.45
9,2,10.000,100.0000,74.50


In [28]:
best_degree_1, best_alpha_1, best_1 = get_best(df1)
print(best_degree_1, best_alpha_1, best_1)

2 0.001 75.65


In [36]:
clf1 = MultiKerOpt(alpha=best_alpha_1, tol=1e-07, degree=best_degree_1, method='svm')
clf1.fit(K_xx_1, y_train_1, n_iter=5)
y_pred_1 = clf1.predict(X_yx_1)

In [37]:
print(y_train_1.mean(), y_pred_1.mean())

0.499 0.521


In [38]:
clf1.method

'svm'

## DATASET 2

In [29]:
K_xx_2, X_yx_2, y_train_2 = build_training(2, sizes=[4,5,6,7], normed=False, rang=(4,4))

counting for size 4 
computing classical training kernel for size 4 
computing classical testing kernel for size 4 
loading special training kernel for size 4 
loading special test kernel for size 4 
counting for size 5 
computing classical training kernel for size 5 
computing classical testing kernel for size 5 
loading special training kernel for size 5 
loading special test kernel for size 5 
counting for size 6 
computing classical training kernel for size 6 
computing classical testing kernel for size 6 
loading special training kernel for size 6 
loading special test kernel for size 6 
counting for size 7 
computing classical training kernel for size 7 
computing classical testing kernel for size 7 
loading special training kernel for size 7 
loading special test kernel for size 7 


In [30]:
RUN = True
if RUN:
    df2  = CvSearch(K_xx_2, X_yx_2, y_train_2, method='svm', degrees=[1, 2, 3], 
               alphas=REG_PARAMS_SPAN, cv=5)
    df2.to_csv('./CV/X2_cv5_linear_kmers_mis.csv', index=False)
else:
    df2 = pd.read_csv('./CV/X2_cv5_linear_kmers_mis.csv')

Iteration 75/75 

Done in 11.094


In [31]:
df2

,degree,alpha,train,val
0,1,0.001,100.00,65.00
1,1,0.010,100.00,65.30
2,1,0.100,95.00,65.55
3,1,1.000,60.90,60.05
4,1,10.000,60.05,59.15
5,2,0.001,100.00,64.55
6,2,0.010,100.00,67.25
7,2,0.100,100.00,64.65
8,2,1.000,100.00,65.85
9,2,10.000,100.00,65.25


In [32]:
best_degree_2, best_alpha_2, best_2 = get_best(df2)
print(best_degree_2, best_alpha_2, best_2)

2 0.01 67.25


In [39]:
clf2 = MultiKerOpt(alpha=best_alpha_2, tol=1e-07, degree=best_degree_2, method='svm')
clf2.fit(K_xx_2, y_train_2, n_iter=5)
y_pred_2 = clf2.predict(X_yx_2)

In [40]:
print(y_train_2.mean(), y_pred_2.mean())

0.4995 0.436


## PREDICTIONS FILES

In [33]:
exp_score = (1/3)*(best_0 + best_1 + best_2)
print(exp_score)

68.86666666666667


In [41]:
y_pred = np.hstack((y_pred_0, y_pred_1, y_pred_2)).astype(int)

In [42]:
y_pred

array([1, 0, 0, ..., 1, 0, 0])

In [43]:
Ids = np.array(range(3000))

In [44]:
predictions = pd.DataFrame({'Id':Ids, 'Bound':y_pred.flatten()})

In [45]:
predictions.head()

,Id,Bound
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0


In [46]:
labels_train.head()

,Id,Bound
0,0,0
1,1,1
2,2,1
3,3,0
4,4,1


In [47]:
predictions.to_csv('predictions10.csv', index=False)